# Code of the IBM Data Science Capstone

## Importation of all the necessary packages

In [1]:
import bs4
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import requests

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from geopy.exc import GeocoderTimedOut

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    cryptography-2.4.2         |   py36h1ba5d50_0         618 KB
    openssl-1.1.1a             |    h14c3975_1000         4.0 MB  conda-forge
    libarchive-3.3.3           |       h5d8350f_5         1.5 MB
    grpcio-1.16.1              |   py36hf8bcb03_1         1.1 MB
    geopy-1.18.1               |             py_0          51 KB  conda-forge
    conda-4.6.2                |           py36_0         869 KB  conda-forge
    libssh2-1.8.0              |                1         239 KB  conda-forge
    python-3.6.8               |       h0371630_0        34.4 MB
    ------------------------------------------------------------
      

## Retrieval of the data concerning the neighborhoods of Denver

The url of the data concerning the neighborhood is selected, then using the Beautifulsoup package the data are retrieved

In [12]:
source=requests.get('https://en.wikipedia.org/wiki/List_of_neighborhoods_in_Denver').text

In [13]:
soup=BeautifulSoup(source, 'lxml')

In [14]:
neighborhoods=[]
for headline in soup('span', {'class' : 'mw-headline'}):
    #print(headline.text)
    links = headline.find_next('ul').find_all('li')
    for link in links:
        #print('*', link.text)
        neighborhoods.append(link.text)
print(neighborhoods)

['Baker', 'Capitol Hill', 'Central Business District', 'Cherry Creek', 'Cheesman Park', 'City Park', 'Congress Park', 'City Park West', 'Civic Center', 'Country Club', 'Lincoln Park', 'North Capitol Hill', 'Speer', 'Union Station', 'Belcaro', 'Cory-Merrill', 'East Colfax', 'Hale', 'Hilltop', 'Indian Creek', 'Lowry', 'Montclair', 'Park Hill', 'Virginia Village', 'Washington Virginia Vale', 'Windsor', 'Clayton', 'Cole', 'Elyria-Swansea', 'Five Points', 'Globeville', 'North Park Hill', 'South Park Hill', 'Skyland', 'Whittier', 'Denver International Airport', 'Gateway / Green Valley Ranch', 'Montbello', 'Northeast Park Hill', 'Stapleton', 'Auraria', 'Berkeley', 'Chaffee Park', 'Highland', 'Jefferson Park', 'Regis', 'Sloan Lake', 'Sunnyside', 'West Highland', 'College View / South Platte', 'Overland', 'Platt Park', 'Rosedale', 'University', 'University Hills', 'University Park', 'Washington Park', 'Washington Park West', 'Wellshire', 'Goldsmith', 'Hampden', 'Hampden South', 'Kennedy', 'Sout

The obtained data contain also names give by the local to the neighborhood then only the part concerning official names is kept

In [15]:
neighbourhoods=neighborhoods[0:79]

Then turned into a DataFrame

In [16]:
df_denver=pd.DataFrame()
df_denver['Neighbourhood']=neighbourhoods

The coordinates of the neighborhood are retrieved using a geolocator <br />  (Please be aware as mentioned in the former assignments that this locator doesn't always work well which explained the modification of the dataframe after retrieval)

In [17]:
def geol_lat_long(address):

    geolocator = Nominatim(user_agent="my-application")
    location = geolocator.geocode(address+',Denver, Colorado')
    if location:
        latitude = location.latitude
        longitude = location.longitude
        return latitude,longitude
    else:
        return('0,0')
    print('Done!')

In [20]:
df_denver['lat-long'] = df_denver['Neighbourhood'].apply(geol_lat_long)

The modifications of the data as mentioned ealier

In [21]:
df_denver.loc[9,'lat-long']='(39.7183, -104.9651)'
df_denver.loc[15,'lat-long']='(39.6863, -104.9436)'
df_denver.loc[19,'lat-long']='(39.6857, -104.8965)'
df_denver.loc[22,'lat-long']='(39.7561, -104.9272)'
df_denver.loc[23,'lat-long']='(39.6898, -104.9330)'
df_denver.loc[24,'lat-long']='(39.7031, -104.9223)'
df_denver.loc[36,'lat-long']='(39.7800, -104.7700)'
df_denver.loc[49,'lat-long']='(39.6630, -105.0216)'
df_denver.loc[55,'lat-long']='(39.6786, -104.9694)'
df_denver.loc[64,'lat-long']='(39.6598, -105.0645)'

The following dataframe is obtained

In [22]:
df_denver

,Neighbourhood,lat-long
0,Baker,"(39.7115945, -104.993749857085)"
1,Capitol Hill,"(39.7358754, -104.9799213)"
2,Central Business District,"(39.74737825, -104.992736894662)"
3,Cherry Creek,"(39.6896651, -104.9106186)"
4,Cheesman Park,"(39.7360266, -104.966401651905)"
5,City Park,"(39.74734, -104.950284973701)"
6,Congress Park,"(39.7337198, -104.9483668)"
7,City Park West,"(39.74537615, -104.966577264397)"
8,Civic Center,"(39.738181, -104.9877439)"
9,Country Club,"(39.7183, -104.9651)"


The following code concerns the splitting of the Latitude and Longitude into two distinct columns

In [23]:
lat_long= df_denver['lat-long'].astype('str')
lat_long=lat_long.str.strip('()') \
    .str.split(',', expand=True) \
    .rename(columns={0:'Latitude', 1:'Longitude'})
df_7=df_denver.assign(Latitude=lat_long['Latitude'].values)
df_8=df_7.assign(Longitude=lat_long['Longitude'].values)
df_8.drop('lat-long', axis=1, inplace=True)
df_denver=df_8
df_denver['Latitude']=df_denver['Latitude'].apply(float)
df_denver['Longitude']=df_denver['Longitude'].apply(float)

In [24]:
df_denver

,Neighbourhood,Latitude,Longitude
0,Baker,39.711594,-104.993750
1,Capitol Hill,39.735875,-104.979921
2,Central Business District,39.747378,-104.992737
3,Cherry Creek,39.689665,-104.910619
4,Cheesman Park,39.736027,-104.966402
5,City Park,39.747340,-104.950285
6,Congress Park,39.733720,-104.948367
7,City Park West,39.745376,-104.966577
8,Civic Center,39.738181,-104.987744
9,Country Club,39.718300,-104.965100


Let's get the coordinates of Denver and then create a map to check the location of the neighborhood <br /> Using the geolocator it could have been possible to end up with neigborhoods around the world

In [25]:
address = 'Denver'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Denver are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Denver are 39.7392364, -104.9848623.


In [26]:
# create map of Denver using latitude and longitude values
map_denver = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_denver['Latitude'], df_denver['Longitude'], df_denver['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_denver)  
    
map_denver

Here everything is fine all neighborhoods are contained in the city

## Retrieval of the data concerning the venues in the neighborhoods of Denver

Let's start first by giving the credentails

In [28]:
CLIENT_ID = 'CHIMM3ACKRW23NBFCFPNP2OBTIGCSDKX1LKDNUH3ZC4DP2C3'
CLIENT_SECRET = 'S0YROIXZHMXTO3BUUCPILAANDIULE2H02B0JDCRL5F1YXUFF'
VERSION = '20180605'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CHIMM3ACKRW23NBFCFPNP2OBTIGCSDKX1LKDNUH3ZC4DP2C3
CLIENT_SECRET:S0YROIXZHMXTO3BUUCPILAANDIULE2H02B0JDCRL5F1YXUFF


Then display the limit of selected venues and the search radius <br /> Please remind also here that the API of Foursquare is limited because it was using the free developper account

In [29]:
LIMIT = 1000

radius = 500

Defines the code to retrieve the venues in Denver

In [30]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT) 
       
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        #results = requests.get(url).json()
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'], #) for v in results]) 
            v['venue']['categories'][0]['name']) for v in results])  

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Gives the list of the retrieved neighborhoods by Foursquare

In [31]:
denver_venues = getNearbyVenues(names=df_denver['Neighbourhood'],
                                   latitudes=df_denver['Latitude'],
                                   longitudes=df_denver['Longitude']
                                  )

Baker
Capitol Hill
Central Business District
Cherry Creek
Cheesman Park
City Park
Congress Park
City Park West
Civic Center
Country Club
Lincoln Park
North Capitol Hill
Speer
Union Station
Belcaro
Cory-Merrill
East Colfax
Hale
Hilltop
Indian Creek
Lowry
Montclair
Park Hill
Virginia Village
Washington Virginia Vale
Windsor
Clayton
Cole
Elyria-Swansea
Five Points
Globeville
North Park Hill
South Park Hill
Skyland
Whittier
Denver International Airport
Gateway / Green Valley Ranch
Montbello
Northeast Park Hill
Stapleton
Auraria
Berkeley
Chaffee Park
Highland
Jefferson Park
Regis
Sloan Lake
Sunnyside
West Highland
College View / South Platte
Overland
Platt Park
Rosedale
University
University Hills
University Park
Washington Park
Washington Park West
Wellshire
Goldsmith
Hampden
Hampden South
Kennedy
Southmoor Park
Bear Valley
Fort Logan
Harvey Park
Harvey Park South
Marston
Athmar Park
Barnum
Barnum West
Mar Lee
Ruby Hill
Sun Valley
Valverde
Villa Park
West Colfax
Westwood


In [32]:
print(denver_venues.shape)
denver_venues.head()

(1309, 7)


,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Baker,39.711594,-104.99375,IMT Alameda Station,39.711561,-104.992922,Residential Building (Apartment / Condo)
1,Baker,39.711594,-104.99375,Stranahan's Colorado Whiskey,39.712253,-104.998576,Distillery
2,Baker,39.711594,-104.99375,Stranahan's Whiskey Tour,39.712208,-104.998531,Whisky Bar
3,Baker,39.711594,-104.99375,The White Whale Room,39.708428,-104.992812,Bar
4,Baker,39.711594,-104.99375,Starbucks,39.710423,-104.988667,Coffee Shop


The data are then group by neighborhoods

In [33]:
denver_venues.groupby('Neighbourhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Athmar Park,3,3,3,3,3,3
Auraria,25,25,25,25,25,25
Baker,10,10,10,10,10,10
Barnum,6,6,6,6,6,6
Barnum West,1,1,1,1,1,1
Bear Valley,3,3,3,3,3,3
Belcaro,6,6,6,6,6,6
Berkeley,34,34,34,34,34,34
Capitol Hill,49,49,49,49,49,49


To make the analysis simplier the data are changed to dummies

In [34]:
# one hot encoding
denver_onehot = pd.get_dummies(denver_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
denver_onehot['Neighbourhood'] = denver_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [denver_onehot.columns[-1]] + list(denver_onehot.columns[:-1])
denver_onehot = denver_onehot[fixed_columns]

denver_onehot.head()

,Neighbourhood,ATM,Accessories Store,Airport,Airport Lounge,Airport Service,Airport Terminal,Alternative Healer,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Assisted Living,Athletics & Sports,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Event Space,Exhibit,Factory,Farm,Farmers Market,Fast Food Restaurant,Financial or Legal Service,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Service,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Golf Course,Government Building,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hawaiian Restaurant,Health & Beauty Service,Historic Site,History Museum,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Insurance Office,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Marijuana Dispensary,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Mongolian Restaurant,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,Opera House,Optical Shop,Organic Grocery,Other Repair Shop,Outdoor Sculpture,Paper / Office Supplies Store,Park,Parking,Pedestrian Plaza,Performing Arts Venue,Pet Store,Pharmacy,Piano Bar,Pizza Place,Playground,Plaza,Pool,Pub,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Club,Rugby Pitch,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Sushi Restaurant,Taco Place,Tailor Shop,Tanning Salon,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Toy / Game Store,Track,Trade School,Trail,Train Station,Travel Lounge,Vacation Rental,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Volleyball Court,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Baker,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Baker,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

And then group by neighborhoods again

In [35]:
denver_grouped = denver_onehot.groupby('Neighbourhood').mean().reset_index()
denver_grouped.head()

,Neighbourhood,ATM,Accessories Store,Airport,Airport Lounge,Airport Service,Airport Terminal,Alternative Healer,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Assisted Living,Athletics & Sports,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Event Space,Exhibit,Factory,Farm,Farmers Market,Fast Food Restaurant,Financial or Legal Service,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Service,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Golf Course,Government Building,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hawaiian Restaurant,Health & Beauty Service,Historic Site,History Museum,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Insurance Office,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Marijuana Dispensary,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Mongolian Restaurant,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,Opera House,Optical Shop,Organic Grocery,Other Repair Shop,Outdoor Sculpture,Paper / Office Supplies Store,Park,Parking,Pedestrian Plaza,Performing Arts Venue,Pet Store,Pharmacy,Piano Bar,Pizza Place,Playground,Plaza,Pool,Pub,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Club,Rugby Pitch,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Sushi Restaurant,Taco Place,Tailor Shop,Tanning Salon,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Toy / Game Store,Track,Trade School,Trail,Train Station,Travel Lounge,Vacation Rental,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Volleyball Court,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Athmar Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.000000,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,

Let's check the number of neighborhoods (here 77)

In [36]:
denver_grouped.shape

(76, 242)

To see which venues are most representative of a neighborhoods the top 5 is made

In [37]:
num_top_venues = 5

for hood in denver_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = denver_grouped[denver_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Athmar Park----
             venue  freq
0             Park  0.33
1     Home Service  0.33
2  Vacation Rental  0.33
3              ATM  0.00
4      Music Venue  0.00


----Auraria----
                  venue  freq
0           Coffee Shop  0.16
1   American Restaurant  0.12
2  Fast Food Restaurant  0.08
3            Sports Bar  0.08
4               Stadium  0.08


----Baker----
                           venue  freq
0                   Intersection   0.1
1                     Whisky Bar   0.1
2  Paper / Office Supplies Store   0.1
3                            Bar   0.1
4                     Food Truck   0.1


----Barnum----
               venue  freq
0   Insurance Office  0.17
1  Convenience Store  0.17
2               Food  0.17
3         Hookah Bar  0.17
4  Other Repair Shop  0.17


----Barnum West----
          venue  freq
0         Hotel   1.0
1           ATM   0.0
2          Park   0.0
3   Music Venue   0.0
4  Neighborhood   0.0


----Bear Valley----
          venue  freq
0    

Then the function to obtain the most common venues is defined

In [38]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Let's use 5 top venues per neighborhood

In [39]:
num_top_venues = 3

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = denver_grouped['Neighbourhood']

for ind in np.arange(denver_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(denver_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Athmar Park,Park,Vacation Rental,Home Service
1,Auraria,Coffee Shop,American Restaurant,Fast Food Restaurant
2,Baker,Paper / Office Supplies Store,Whisky Bar,Distillery
3,Barnum,Other Repair Shop,Food,Convenience Store
4,Barnum West,Hotel,Zoo Exhibit,Dive Bar
5,Bear Valley,Park,Playground,Zoo Exhibit
6,Belcaro,Sushi Restaurant,Bakery,Lawyer
7,Berkeley,Coffee Shop,Italian Restaurant,Breakfast Spot
8,Capitol Hill,Pizza Place,Sandwich Place,Pub
9,Central Business District,Hotel,American Restaurant,Sandwich Place


A machine learning method is used to separate the neigborhood into categories <br /> Here 3 categories are used after trying different setups

In [40]:
# set number of clusters
kclusters = 3

denver_grouped_clustering = denver_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, init='k-means++', random_state=0, n_init=12).fit(denver_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 1, 0, 1, 1, 1, 0, 1], dtype=int32)

In [41]:
denver_merged = df_denver
denver_merged=denver_merged.drop(denver_merged.index[68]).drop(denver_merged.index[23]).drop(denver_merged.index[19]).reset_index(drop=True)
# add clustering labels


denver_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighbourhood
denver_merged = denver_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

denver_merged.head() # check the last columns!

,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Baker,39.711594,-104.993750,0,Paper / Office Supplies Store,Whisky Bar,Distillery
1,Capitol Hill,39.735875,-104.979921,1,Pizza Place,Sandwich Place,Pub
2,Central Business District,39.747378,-104.992737,1,Hotel,American Restaurant,Sandwich Place
3,Cherry Creek,39.689665,-104.910619,1,Park,Rugby Pitch,Gym / Fitness Center
4,Cheesman Park,39.736027,-104.966402,1,Mexican Restaurant,Coffee Shop,Park


A map of the categories is created

In [42]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(denver_merged['Latitude'], denver_merged['Longitude'], denver_merged['Neighbourhood'], denver_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-2],
        fill=True,
        fill_color=rainbow[cluster-2],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Let's have a closer look to each categories

In [61]:
denver_merged.loc[denver_merged['Cluster Labels'] == 0, denver_merged.columns[[0] + list(range(4, denver_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Baker,Paper / Office Supplies Store,Whisky Bar,Distillery
5,City Park,Zoo Exhibit,Park,Exhibit
10,Lincoln Park,Steakhouse,Café,Intersection
12,Speer,Coffee Shop,Yoga Studio,Rental Car Location
17,Hale,Coffee Shop,Auto Workshop,Gym
29,North Park Hill,Coffee Shop,Scenic Lookout,Park
33,Denver International Airport,Airport Service,Airport Lounge,Train Station
34,Gateway / Green Valley Ranch,ATM,Pharmacy,Bank
43,Regis,Liquor Store,Golf Course,Convenience Store
45,Sunnyside,Coffee Shop,Mexican Restaurant,Flower Shop


In [62]:
denver_merged.loc[denver_merged['Cluster Labels'] == 1, denver_merged.columns[[0] + list(range(4, denver_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
1,Capitol Hill,Pizza Place,Sandwich Place,Pub
2,Central Business District,Hotel,American Restaurant,Sandwich Place
3,Cherry Creek,Park,Rugby Pitch,Gym / Fitness Center
4,Cheesman Park,Mexican Restaurant,Coffee Shop,Park
6,Congress Park,Coffee Shop,Arts & Crafts Store,Gift Shop
7,City Park West,American Restaurant,Brewery,Pizza Place
8,Civic Center,Sandwich Place,Hotel,Bar
9,Country Club,Golf Course,Salon / Barbershop,Shipping Store
11,North Capitol Hill,American Restaurant,Coffee Shop,Asian Restaurant
13,Union Station,Hotel,Restaurant,American Restaurant


In [64]:
denver_merged.loc[denver_merged['Cluster Labels'] == 2, denver_merged.columns[[0] + list(range(4, denver_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
26,Elyria-Swansea,Food,Mexican Restaurant,Farm


## Retrieval of data concerning the breweries in Denver

Let's get the coordinates of Denver once again

In [43]:
address = 'Denver, CO'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

39.7392364 -104.9848623


The search query is specify to concerne only the breweries in Denver

In [4]:
search_query = 'brewery'
radius_brew = 6500
limit_brew=1000
print(search_query + ' .... OK!')

brewery .... OK!


The url sent to the Foursquare API is created

In [7]:
url2 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius_brew, limit_brew)
url2

'https://api.foursquare.com/v2/venues/search?client_id=CHIMM3ACKRW23NBFCFPNP2OBTIGCSDKX1LKDNUH3ZC4DP2C3&client_secret=S0YROIXZHMXTO3BUUCPILAANDIULE2H02B0JDCRL5F1YXUFF&ll=39.7392364,-104.9848623&v=20180605&query=brewery&radius=6500&limit=1000'

And the request is sent

In [8]:
results = requests.get(url2).json()

The received data are turned into a DataFrame

In [9]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)

Then only the interesting columns of the dataframe are kept

In [10]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Rock Bottom Brewery,Brewery,1001 16th St # A-100,US,Denver,United States,btwn Curtis St and Arapahoe St,1254,[1001 16th St # A-100 (btwn Curtis St and Arap...,"[{'label': 'display', 'lat': 39.7475786, 'lng'...",39.747579,-104.994722,NaN,80265,CO,40e0b100f964a520ed031fe3
1,Jagged Mountain Brewery,Brewery,1139 20th St,US,Denver,United States,Lawrence Street,1546,"[1139 20th St (Lawrence Street), Denver, CO 80...","[{'label': 'display', 'lat': 39.75214633776, '...",39.752146,-104.991537,NaN,80202,CO,522132a711d261ae095938b9
2,Denver ChopHouse & Brewery,Steakhouse,1735 19th St Ste 100,US,Denver,United States,at Wynkoop,2055,"[1735 19th St Ste 100 (at Wynkoop), Denver, CO...","[{'label': 'display', 'lat': 39.75507154716135...",39.755072,-104.997223,NaN,80202,CO,4ae39034f964a520b79621e3
3,Blue Moon Brewery at The Sandlot,Brewery,2161 Blake St,US,Denver,United States,NaN,1983,"[2161 Blake St, Denver, CO 80205, United States]","[{'label': 'display', 'lat': 39.75608916394047...",39.756089,-104.992384,NaN,80205,CO,40e0b100f964a520f4031fe3
4,Bull & Bush Pub & Brewery,Pub,4700 E Cherry Creek South Dr,US,Denver,United States,at Dexter St,6023,"[4700 E Cherry Creek South Dr (at Dexter St), ...","[{'label': 'display', 'lat': 39.70309851200205...",39.703099,-104.932501,NaN,80246,CO,40e0b100f964a52024031fe3
5,Lowdown Brewery+Kitchen,Brewery,800 N Lincoln St,US,Denver,United States,8th Street,1112,"[800 N Lincoln St (8th Street), Denver, CO 802...","[{'label': 'display', 'lat': 39.72927815753679...",39.729278,-104.985969,NaN,80203,CO,5238b84911d2b81d3be80b22
6,Liberati Restaurant & Brewery,Brewery,2403 Champa St,US,Denver,United States,24th St,1673,"[2403 Champa St (24th St), Denver, CO 80205, U...","[{'label': 'display', 'lat': 39.754272, 'lng':...",39.754272,-104.984538,Curtis Park,80205,CO,5a21abc48173cb3fac6c2607
7,Breckenridge Brewery Mountain House,American Restaurant,2220 Blake St,US,Denver,United States,NaN,2127,"[2220 Blake St, Denver, CO, United States]","[{'label': 'display', 'lat': 39.7480775898278,...",39.748078,-105.006897,Auraria,NaN,CO,582fb1bc04aa4f5e813ca768
8,Vine Street Pub & Brewery,Brewery,1700 Vine St,US,Denver,United States,at 17th Ave,2008,"[1700 Vine St (at 17th Ave), Denver, CO 80206,...","[{'label': 'display', 'lat': 39.74333520746895...",39.743335,-104.962009,NaN,80206,CO,49dfe1d2f964a52024611fe3
9,Brewery Bar,Mexican Restaurant,150 Kalamath St,US,Denver,United States,NaN,2563,"[150 Kalamath St, Denver, CO 80223, United Sta...","[{'label': 'display', 'lat': 39.71934924007873...",39.719349,-104.999968,NaN,80223,CO,4ec2b7fee5fabc46cc5c4e2b


To see wher the breweries in Denver are located a map is created

In [52]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=11) 

# add a red circle marker to represent the Conrad Hotel
#folium.features.CircleMarker(
#    [latitude, longitude],
 #   radius=10,
  #  color='red',
   # popup='Conrad Hotel',
    #fill = True,
    #fill_color = 'red',
    #fill_opacity = 0.6
#).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)
    
# display map
venues_map

In [60]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=11) 

# add a red circle marker to represent the Conrad Hotel
#folium.features.CircleMarker(
#    [latitude, longitude],
 #   radius=10,
  #  color='red',
   # popup='Conrad Hotel',
    #fill = True,
    #fill_color = 'red',
    #fill_opacity = 0.6
#).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.Marker(
        [lat, lng],
       
        popup=label,
       
    ).add_to(venues_map)
    
markers_colors = []
for lat, lon, poi, cluster in zip(denver_merged['Latitude'], denver_merged['Longitude'], denver_merged['Neighbourhood'], denver_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(venues_map)

# display map
venues_map

## Retrieval of data concerning the average income according to the neighborhood

The data used here come from the US Census which took place in 2016 <br /> The data are uploaded from a csv file

In [61]:
denver_income=pd.read_csv('denver_income_2.csv', sep=';')

The column is renamed to match the rest of the assignment

In [62]:
denver_income.rename(columns={'Neighborhood': 'Neighbourhood'}, inplace=True)

Let's have a look

In [63]:
denver_income.head()

,Neighbourhood,Average Income
0,Alder-Three Sisters Park,105682.45
1,Allendale,65216.87
2,Alta Vista and Memorial Park,68761.63
3,"Apel Bacher Park, Koch Sub and Coulehan Grange",62294.80
4,"Applewood, Echo Hill, Rolling Hills and Meadows",125926.64


The Census data are then emerged with the data containing the coordinates of the neighborhoods

In [64]:
denver_income_map = pd.merge(denver_income, df_denver, on='Neighbourhood', how='right')

The column "Average Income" is changed to a float to better handling

In [65]:
denver_income['Average Income']=denver_income['Average Income'].apply(float)

Let's check the new DataFrame

In [66]:
denver_income_map.head()

,Neighbourhood,Average Income,Latitude,Longitude
0,Athmar Park,55345.87,39.703682,-105.010741
1,Auraria,96844.19,39.746522,-105.006710
2,Baker,75913.14,39.711594,-104.993750
3,Barnum,45686.08,39.717683,-105.032455
4,Barnum West,60091.15,39.717155,-105.046497


A map of the average income according to the neighborhood is created, the breweries are also added <br /> The GeoJson is obtained on https://www.denvergov.org/opendata after convertion from a shape file

In [68]:
#world_geo = r'world_countries.json' # geojson file
neighborhood_geo=r'statistical_neighborhoods.geojson'

income_map = folium.Map(location=[latitude, longitude], zoom_start=12) 

income_map.choropleth(
    geo_data=neighborhood_geo,
    name='choropleth',
    data=denver_income_map,
    key_on='feature.properties.NBHD_NAME',
    columns=['Neighbourhood', 'Average Income'],
    fill_color='YlGnBu',
    fill_opacity=0.7,
    line_opacity=0.8,
    legend_name='Average Income'
)

for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(income_map)

income_map